# 可视化监控训练过程
## tensorboardX
* pytorch 1.8 之后自带tensorboardx `torch.utils.tensorboard`
* `pip install tensorboardX`
* 使用
  * `torch.utils.tensorboard.writer.SummaryWriter(log_dir=None)`
    * 新建writer实例，log_dir为日志保存的路径，默认在runs/当前时间_主机名 文件夹
  * `add_scalar(tag, scalar_value, global_step=None, walltime=None)`
    * tag 字符串，表明数据名称，不同名称不同曲线
    * scalar_value 浮点，要保存的数据
    * global_step 训练的step
    * walltime 浮点数，记录发生的时间 默认time.time()
  * `add_image(tag, img_tensor, global_step=None, walltime=None, dataformats='CHW')`
    * img_tensor tensor类型或者numpy中的array类型，表示图像
    * dataformats 字符串，默认CHW channel x height x width 或者CHW,HWC,HW等

In [2]:
#使用
from torch.utils.tensorboard import SummaryWriter
import numpy as np

writer  = SummaryWriter()

for n_iter in range(100):
    writer.add_scalar('Loss/train',np.random.random(), n_iter)
    writer.add_scalar('Loss/test',np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train',np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test',np.random.random(), n_iter)
img = np.zeros((3, 100, 100))

img[0] = np.arange(0,10000).reshape(100,100)/10000
img[0] = 1- np.arange(0,10000).reshape(100,100)/10000

writer.add_image('my_image', img, 0)
writer.close()

## 之后在本地运行 tensorboard --logdir=runs

In [4]:
# 训练过程可视化
import random
import numpy as np
import torch
from torch import nn

class LinearModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(1))
        self.bias = nn.Parameter(torch.randn(1))
    def forward(self, input):
        return (input*self.weight)+self.bias
w =2
b=3
xlim = [-10, 10]
x_train = np.random.randint(low=xlim[0], high=xlim[1], size=30)
y_train = [w*x+b +random.randint(0,2) for x in x_train]

from torch.utils.tensorboard import SummaryWriter

model = LinearModel()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, weight_decay=1e-2, momentum=0.9)

y_train=torch.tensor(y_train,dtype=torch.float32)

writer = SummaryWriter()

for n_iter in range(500):
    input = torch.from_numpy(x_train)
    output = model(input)
    loss = nn.MSELoss()(output, y_train)
    model.zero_grad()
    loss.backward()
    optimizer.step()
    writer.add_scalar('Loss/Train',loss,n_iter)

## visdom
* 实时显示曲线
* `pythom -m visdom.server`
* 先启动visdom后，再查看 `127.0.0.1:8097`

In [5]:

from visdom import Visdom
import numpy as np
import time

# 将窗口类实例化
viz = Visdom() 
# 创建窗口并初始化
viz.line([0.], [0], win='train_loss', opts=dict(title='train_loss'))
# viz.line([[0., 0.]], [0], win='loss and accuracy', opts=dict(title='train_loss'))

for n_iter in range(10):
    # 随机获取loss值
    loss = 0.2 * np.random.randn() + 1
    # 更新窗口图像
    viz.line([loss], [n_iter], win='train_loss', update='append')
    # viz.line(Y=[[loss, accuracy]], X=[n_iter], win='loss and accuracy', update='append')
    time.sleep(0.5)

img = np.zeros((3, 100, 100))
img[0] = np.arange(0, 10000).reshape(100, 100) / 10000
img[1] = 1 - np.arange(0, 10000).reshape(100, 100) / 10000
# 可视化图像
viz.image(img)

Setting up a new session...


'window_3b237e82fbaf9a'

In [12]:
from visdom import Visdom

import random
import numpy as np
import torch
from torch import nn
import time


class LinearModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(1))
        self.bias = nn.Parameter(torch.randn(1))
    def forward(self, input):
        return (input*self.weight)+self.bias
w =2
b=3
xlim = [-10, 10]
x_train = np.random.randint(low=xlim[0], high=xlim[1], size=30)
y_train = [w*x+b +random.randint(0,2) for x in x_train]

from torch.utils.tensorboard import SummaryWriter

model = LinearModel()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, weight_decay=1e-2, momentum=0.9)

y_train=torch.tensor(y_train,dtype=torch.float32)

# writer = SummaryWriter()
viz = Visdom(port=8097)
viz.line([0.],[0],win='train_loss',opts=dict(title='train_loss'))

for n_iter in range(500):
    input = torch.from_numpy(x_train)
    output = model(input)
    loss = nn.MSELoss()(output, y_train)
    model.zero_grad()
    loss.backward()
    optimizer.step()
    viz.line([loss.item()],[n_iter],win='train_loss',update='append')
    # writer.add_scalar('Loss/Train',loss,n_iter)


Setting up a new session...
